In [20]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [21]:
mice_eeg1 = pd.read_csv('EEG_mouse_data_1.csv')
mice_eeg2 = pd.read_csv('EEG_mouse_data_2.csv')
mice_eeg = pd.concat([mice_eeg1, mice_eeg2])
mice_eeg

,state,amplitude_around_1_Hertz,amplitude_around_2_Hertz,amplitude_around_3_Hertz,amplitude_around_4_Hertz,amplitude_around_5_Hertz,amplitude_around_6_Hertz,amplitude_around_7_Hertz,amplitude_around_8_Hertz,amplitude_around_9_Hertz,...,amplitude_around_92_Hertz,amplitude_around_93_Hertz,amplitude_around_94_Hertz,amplitude_around_95_Hertz,amplitude_around_96_Hertz,amplitude_around_97_Hertz,amplitude_around_98_Hertz,amplitude_around_99_Hertz,amplitude_around_100_Hertz,amplitude_around_101_Hertz
0,n,0.000009,0.000014,0.000013,0.000014,0.000023,0.000009,0.000019,0.000011,0.000016,...,8.364060e-09,4.208368e-09,2.177679e-09,4.342505e-09,5.130010e-09,2.852361e-09,9.772395e-10,1.876307e-09,5.457793e-09,8.653900e-08
1,n,0.000010,0.000010,0.000011,0.000015,0.000022,0.000017,0.000027,0.000011,0.000011,...,6.695745e-09,1.213822e-08,2.287185e-08,3.219380e-09,2.461736e-09,2.985953e-09,1.434412e-09,1.579590e-09,5.458902e-09,8.492570e-08
2,n,0.000011,0.000077,0.000034,0.000019,0.000013,0.000006,0.000010,0.000004,0.000006,...,1.105705e-08,3.264479e-09,1.204057e-09,5.090074e-09,5.477467e-09,1.809223e-09,7.064289e-10,1.435585e-09,7.416121e-09,1.156240e-07
3,w,0.000003,0.000031,0.000044,0.000016,0.000007,0.000006,0.000005,0.000007,0.000006,...,2.223992e-08,1.028462e-08,3.561479e-09,3.704753e-09,2.425319e-09,1.536834e-09,2.540238e-10,6.300358e-10,3.647454e-09,2.212800e-08
4,w,0.000010,0.000011,0.000012,0.000017,0.000020,0.000018,0.000014,0.000025,0.000018,...,6.541705e-09,3.312024e-09,8.779863e-09,7.400000e-09,2.890853e-09,5.709757e-10,2.171227e-09,6.757242e-10,2.005702e-09,2.861980e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20600,w,0.000025,0.000007,0.000007,0.000003,0.000005,0.000002,0.000005,0.000010,0.000013,...,1.162051e-08,9.470367e-09,1.542617e-08,6.114029e-09,2.346950e-09,4.264138e-09,1.733832e-09,8.314190e-10,1.720087e-09,1.345590e-09
20601,w,0.000017,0.000038,0.000015,0.000011,0.000019,0.000006,0.000010,0.000012,0.000009,...,1.456810e-08,9.259630e-09,3.831858e-08,3.839020e-09,4.668449e-09,1.671207e-09,2.347928e-09,2.219437e-09,2.045267e-09,6.063210e-09
20602,w,0.000016,0.000028,0.000031,0.000011,0.000019,0.000001,0.000015,0.000002,0.000007,...,1.197483e-08,2.496412e-08,2.292325e-08,2.783310e-09,6.423075e-09,3.649511e-09,4.350072e-09,6.656486e-10,2.086456e-09,2.761580e-08
20603,w,0.000013,0.000014,0.000020,0.000018,0.000007,0.000013,0.000009,0.000018,0.000005,...,3.028127e-08,9.760537e-09,3.475243e-09,2.667755e-09,4.243288e-09,6.048632e-09,3.234623e-09,1.421633e-09,9.709723e-10,2.062290e-08


In [22]:
mice25features = mice_eeg.iloc[:,:26]
mice25features

,state,amplitude_around_1_Hertz,amplitude_around_2_Hertz,amplitude_around_3_Hertz,amplitude_around_4_Hertz,amplitude_around_5_Hertz,amplitude_around_6_Hertz,amplitude_around_7_Hertz,amplitude_around_8_Hertz,amplitude_around_9_Hertz,...,amplitude_around_16_Hertz,amplitude_around_17_Hertz,amplitude_around_18_Hertz,amplitude_around_19_Hertz,amplitude_around_20_Hertz,amplitude_around_21_Hertz,amplitude_around_22_Hertz,amplitude_around_23_Hertz,amplitude_around_24_Hertz,amplitude_around_25_Hertz
0,n,0.000009,0.000014,0.000013,0.000014,0.000023,0.000009,0.000019,0.000011,0.000016,...,1.121431e-06,9.572615e-07,7.912338e-07,2.075210e-06,2.125421e-06,9.183682e-07,1.442768e-06,1.315617e-06,1.882535e-06,3.555321e-07
1,n,0.000010,0.000010,0.000011,0.000015,0.000022,0.000017,0.000027,0.000011,0.000011,...,1.214325e-06,2.456797e-06,2.102875e-06,3.754975e-07,1.144381e-06,1.869306e-06,9.546215e-07,8.958666e-07,2.865039e-07,8.280935e-07
2,n,0.000011,0.000077,0.000034,0.000019,0.000013,0.000006,0.000010,0.000004,0.000006,...,2.251707e-06,3.322518e-06,1.112426e-06,8.238182e-07,8.392117e-07,6.979253e-07,4.631708e-07,3.179654e-07,3.833318e-07,2.466324e-07
3,w,0.000003,0.000031,0.000044,0.000016,0.000007,0.000006,0.000005,0.000007,0.000006,...,2.197342e-06,8.276670e-07,6.019235e-07,3.639088e-07,3.230235e-07,4.536860e-07,5.793950e-07,2.854656e-07,9.187695e-07,5.579241e-07
4,w,0.000010,0.000011,0.000012,0.000017,0.000020,0.000018,0.000014,0.000025,0.000018,...,3.330912e-06,3.003597e-06,3.560957e-06,1.856221e-06,8.180798e-07,1.657310e-06,2.038410e-06,9.678699e-07,6.374025e-07,8.643028e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20600,w,0.000025,0.000007,0.000007,0.000003,0.000005,0.000002,0.000005,0.000010,0.000013,...,7.605994e-07,5.857975e-07,2.008976e-06,1.047500e-06,8.361965e-07,1.187530e-06,1.362517e-06,1.155722e-06,4.674335e-07,4.710592e-07
20601,w,0.000017,0.000038,0.000015,0.000011,0.000019,0.000006,0.000010,0.000012,0.000009,...,4.660933e-07,4.797065e-07,1.377159e-06,1.549855e-06,2.014634e-06,1.006481e-06,1.255027e-06,7.684788e-07,1.131635e-06,1.080890e-06
20602,w,0.000016,0.000028,0.000031,0.000011,0.000019,0.000001,0.000015,0.000002,0.000007,...,9.727383e-07,5.698932e-07,8.794702e-07,9.412227e-07,7.479130e-07,1.173636e-07,8.319340e-07,1.222192e-07,1.277335e-07,2.256663e-07
20603,w,0.000013,0.000014,0.000020,0.000018,0.000007,0.000013,0.000009,0.000018,0.000005,...,7.172008e-07,1.828871e-06,5.327440e-07,5.149679e-07,6.310218e-07,4.164332e-07,3.309523e-07,8.170078e-07,6.630207e-07,9.615590e-07


In [26]:
# Instanciez un objet StandardScaler
scaler = StandardScaler()


# Normalisez les données
normalized_data = scaler.fit_transform(mice25features.iloc[:,1:26])

# Créez un nouveau DataFrame à partir des données normalisées
normalized_df = pd.DataFrame(normalized_data, columns=mice25features.iloc[:,1:26].columns)

normalize_pd = pd.concat([mice25features.state, normalized_df], axis=1)
print("Données normalisées :")
print(normalized_df)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects